In [1]:
import pandas as pd
import h2o

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [3]:
import multiprocessing
print("CPU: ",multiprocessing.cpu_count())

import psutil
print("Memory: ",psutil.virtual_memory())

CPU:  72
Memory:  svmem(total=1622764929024, available=1422414950400, percent=12.3, used=194895474688, free=556070727680, active=910295629824, inactive=111481761792, buffers=0, cached=871798726656, shared=2571857920, slab=15486603264)


In [4]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 100, nthreads = 5)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O cluster uptime:,12 mins 05 secs
H2O cluster timezone:,Europe/Istanbul
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_96273_ugsf64
H2O cluster total nodes:,1
H2O cluster free memory:,88.3 Gb
H2O cluster total cores:,72
H2O cluster allowed cores:,5
H2O cluster status:,"locked, healthy"


In [5]:
hf_positive = h2o.import_file('dataset/train_true_positive_features.csv')
hf_negative = h2o.import_file('dataset/train_true_negative_features.csv')
hf = hf_positive.rbind(hf_negative)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
print("positive instances: ",hf_positive.shape)
print("negative instances: ",hf_negative.shape)
print("total instances: ",hf.shape)

positive instances:  (165179, 34)
negative instances:  (283367, 34)
total instances:  (448546, 34)


In [7]:
hf = hf[['vgg_cosine', 'vgg_euclidean_l2'
         , 'facenet_cosine', 'facenet_euclidean_l2'
         , 'openface_cosine', 'openface_euclidean_l2'
         , 'is_related']]

In [8]:
hf.head()

vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
0.618396,1.11211,1.25131,1.58197,1.12544,1.50029,1
0.601191,1.09653,1.14205,1.51133,1.08315,1.47183,1
0.543063,1.04217,1.10449,1.48627,1.14981,1.51645,1
0.618544,1.11224,1.24833,1.58008,1.09367,1.47897,1
0.60665,1.1015,1.15115,1.51733,1.11618,1.49411,1
0.630702,1.12312,1.22153,1.56303,1.20384,1.55167,1
0.742856,1.2189,1.09549,1.48019,1.07032,1.4631,1
0.558462,1.05685,1.11984,1.49655,1.24634,1.57882,1
0.648851,1.13917,1.10773,1.48844,0.612283,1.1066,1
0.677681,1.1642,1.03272,1.43716,0.571711,1.06931,1


In [9]:
#convert target label to factor because this is a binary classification
#otherwise, there would be a regression problem
hf['is_related'] = hf['is_related'].asfactor()

In [10]:
#70% train, 15% test, 15% validation
train, test, validation = hf.split_frame(ratios=[0.70, 0.15], seed=17)

In [11]:
print("train set size: ",train.shape)
print("test set size: ", test.shape)
print("validation set size: ", validation.shape)

train set size:  (314122, 7)
test set size:  (67160, 7)
validation set size:  (67264, 7)


# Training

In [12]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [13]:
model = H2OGradientBoostingEstimator(
    ntrees = 1000
    , learn_rate = 0.01
    , stopping_rounds = 50
    , stopping_metric = "AUC"
)

In [14]:
model.train(x = hf.names[0:-1], y = hf.names[-1]
    , training_frame = train
    , validation_frame = test
    #, verbose = True
    , model_id = "GBM_Kinship"
)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [15]:
model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_Kinship


Model Summary: 

,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,1000.0,1000.0,268761.0,0.0,5.0,2.695,1.0,32.0,16.503




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.19740943929831983
RMSE: 0.44430782043344663
LogLoss: 0.5789539838842188
Mean Per-Class Error: 0.3333046491681597
AUC: 0.7276208150873205
pr_auc: 0.6123781904410922
Gini: 0.4552416301746409

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3082383667488498: 

,,0,1,Error,Rate
0,0,110415.0,87952.0,0.4434,(87952.0/198367.0)
1,1,27576.0,88179.0,0.2382,(27576.0/115755.0)
2,Total,137991.0,176131.0,0.3678,(115528.0/314122.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010006,0.828643,2.369794,2.369794,0.870536,0.863653,0.870536,0.863653,0.023712,0.023712,136.979363,136.979363
1,,2,0.020116,0.785004,2.241124,2.305126,0.823270,0.804792,0.846780,0.834070,0.022658,0.046370,124.112442,130.512568
2,,3,0.030003,0.762510,2.086765,2.233170,0.766566,0.772705,0.820347,0.813848,0.020632,0.067002,108.676545,123.316975
3,,4,0.040009,0.741507,2.057872,2.189329,0.755952,0.752465,0.804243,0.798497,0.020591,0.087593,105.787207,118.932902
4,,5,0.050000,0.717152,1.983857,2.148271,0.728763,0.729365,0.789160,0.784683,0.019821,0.107414,98.385658,114.827125
5,,6,0.100045,0.637059,1.882312,2.015232,0.691461,0.673227,0.740289,0.728930,0.094200,0.201613,88.231172,101.523211
6,,7,0.150030,0.576325,1.650202,1.893616,0.606196,0.605305,0.695613,0.687742,0.082486,0.284099,65.020161,89.361565
7,,8,0.200000,0.526255,1.507120,1.797049,0.553635,0.552535,0.660140,0.653960,0.075311,0.359410,50.711951,79.704917
8,,9,0.300045,0.454845,1.333359,1.642440,0.489805,0.488514,0.603345,0.598795,0.133395,0.492805,33.335924,64.243984
9,,10,0.400000,0.395776,1.150853,1.519598,0.422762,0.426238,0.558219,0.555675,0.115034,0.607839,15.085253,51.959791




Scoring History: 

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2019-09-18 12:19:12,0.012 sec,0.0,0.482399,0.658154,0.500000,0.000000,1.000000,0.631497,0.482083,0.657531,0.500000,0.000000,1.000000,0.632653
1,,2019-09-18 12:19:13,1.333 sec,1.0,0.481705,0.656718,0.721281,0.569402,2.289071,0.359561,0.481386,0.656089,0.723266,0.570940,2.291923,0.357370
2,,2019-09-18 12:19:13,1.639 sec,2.0,0.481025,0.655313,0.721487,0.569547,2.289071,0.359561,0.480702,0.654678,0.723424,0.571011,2.291923,0.357370
3,,2019-09-18 12:19:14,1.841 sec,3.0,0.480356,0.653936,0.721588,0.589096,2.289071,0.359561,0.480030,0.653294,0.723559,0.589766,2.291923,0.357370
4,,2019-09-18 12:19:14,2.027 sec,4.0,0.479701,0.652590,0.722411,0.590812,2.288996,0.376733,0.479373,0.651944,0.724206,0.591456,2.293831,0.356358
5,,2019-09-18 12:19:14,2.209 sec,5.0,0.479057,0.651270,0.722372,0.590803,2.288996,0.371531,0.478725,0.650617,0.724238,0.591521,2.293831,0.356626
6,,2019-09-18 12:19:14,2.410 sec,6.0,0.478426,0.649978,0.722452,0.590815,2.288996,0.371149,0.478091,0.649321,0.724261,0.591450,2.293831,0.356358
7,,2019-09-18 12:19:14,2.603 sec,7.0,0.477805,0.648711,0.722552,0.590854,2.292176,0.376284,0.477467,0.648047,0.724298,0.591508,2.304677,0.356313
8,,2019-09-18 12:19:15,2.801 sec,8.0,0.477197,0.647471,0.722574,0.590964,2.292660,0.375908,0.476856,0.646803,0.724324,0.591713,2.304677,0.356313
9,,2019-09-18 12:19:15,2.995 sec,9.0,0.476599,0.646254,0.722710,0.591004,2.292660,0.376370,0.476255,0.645579,0.724557,0.591808,2.304677,0.356313



See the whole table with table.as_data_frame()

Variable Importances: 

,variable,relative_importance,scaled_importance,percentage
0,facenet_euclidean_l2,261705.156250,1.000000,0.468295
1,facenet_cosine,162436.656250,0.620686,0.290664
2,vgg_euclidean_l2,73451.710938,0.280666,0.131434
3,vgg_cosine,34810.042969,0.133012,0.062289
4,openface_cosine,15869.265625,0.060638,0.028396
5,openface_euclidean_l2,10574.509766,0.040406,0.018922


# Model Performance

In [16]:
val_perf = model.model_performance(validation)

In [17]:
val_perf.auc()

0.7225285769297011

In [18]:
val_perf.accuracy()[0][1]

0.6940562559467174

In [19]:
test_perf = model.model_performance(test)

In [20]:
test_perf.auc()

0.727593424555288

In [21]:
test_perf.accuracy()[0][1]

0.6983918999404407

# Predictions

In [22]:
predictions = model.predict(test_data = validation)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [23]:
predictions.tail()

predict,p0,p1
1,0.462883,0.537117
0,0.815415,0.184585
1,0.48551,0.51449
1,0.322137,0.677863
1,0.4286,0.5714
0,0.85153,0.14847
0,0.909476,0.0905237
1,0.336492,0.663508
1,0.631113,0.368887
0,0.857651,0.142349


In [24]:
predictions_pd = predictions.as_data_frame()

In [25]:
predictions_pd.iloc[0]

predict    0.000000
p0         0.886282
p1         0.113718
Name: 0, dtype: float64

# Store model

In [26]:
saved_model = h2o.save_model(model, path = "", force=True)

In [27]:
saved_model

'/outputs/sefik/kinship/GBM_Kinship'

# Load the best model

In [28]:
#you can directly load the best model by running the load_model command
restored_model = h2o.load_model(saved_model)